**Завдання**
### Використовуючи з лекції датасет і код:
* створити на основі двошарової нейромережі тришарову;
* оцінити її похибку та точність визначення класів.
#### Після цього використати для оцінки похибки **Mean Squared Error**, в ролі оптимізатора - **Adam**.
* Порівняти, чи змінилась похибка та точність.

In [3]:
%matplotlib inline
import torch.nn as nn
from tensorflow.keras.datasets import mnist # Біліотека з базою Mnist
from tensorflow.keras.models import Sequential # Підключаємо клас моделі Sequential
from tensorflow.keras.layers import Dense # Підключаємо клас Dense - повнозв'язний шар
from tensorflow.keras.optimizers import Adam # Підключаємо оптимізатор Adam


In [1]:
import sys
print("version:", sys.version)

version: 3.10.5 (tags/v3.10.5:f377153, Jun  6 2022, 16:14:13) [MSC v.1929 64 bit (AMD64)]


In [ ]:
model = Sequential() # Створюємо модель нейронної мережі
model.add(Dense(2, input_dim = 2, use_bias=False)) # Додаємо повнозв'язний шар з 2-ма нейронами (вказуємо, що на вхід приймаємо вектор з двох елементів) і відключаємо використання зміщення нейрона
model.add(Dense(1, use_bias=False))